<a href="https://colab.research.google.com/github/PriyankaKumari-123-82/DAA/blob/main/Neural_network_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Load and Preprocess Data**:
We’ll use the MNIST dataset and handle normalization, flattening, and one-hot encoding manually.

In [8]:
import numpy as np
from tensorflow.keras.datasets import mnist

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flatten images
x_train = x_train.reshape(-1, 28 * 28)
x_test = x_test.reshape(-1, 28 * 28)

# One-hot encode labels
def one_hot_encode(labels, num_classes=10):
    one_hot = np.zeros((labels.shape[0], num_classes))
    one_hot[np.arange(labels.shape[0]), labels] = 1
    return one_hot

y_train = one_hot_encode(y_train)
y_test = one_hot_encode(y_test)


**2. Implement Neural Network**:
Define the components of a neural network, including forward propagation, activation functions, loss function, and backpropagation.

**Initialize Parameters**







In [9]:
def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2


**Activation Functions**

In [10]:
def relu(Z):
    return np.maximum(0, Z)

def relu_derivative(Z):
    return Z > 0

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)


**Forward Propagation**

In [11]:
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    cache = (Z1, A1, Z2, A2)
    return A2, cache


**Compute Loss**

In [12]:
def compute_loss(Y, A2):
    m = Y.shape[0]
    return -np.sum(Y * np.log(A2 + 1e-8)) / m


**Backward Propagation**

In [13]:
def backward_propagation(X, Y, cache, W1, W2):
    Z1, A1, Z2, A2 = cache
    m = X.shape[0]

    dZ2 = A2 - Y
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2


**Update Parameters**

In [14]:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2


**3. Train the Model**:
Implement the training loop to iteratively update weights.

In [15]:
def train(X, Y, input_size, hidden_size, output_size, epochs, learning_rate):
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)

    for epoch in range(epochs):
        A2, cache = forward_propagation(X, W1, b1, W2, b2)
        loss = compute_loss(Y, A2)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, cache, W1, W2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss:.4f}")

    return W1, b1, W2, b2


**4. Evaluate the Model**:
Compute accuracy and visualize predictions.

In [16]:
def predict(X, W1, b1, W2, b2):
    A2, _ = forward_propagation(X, W1, b1, W2, b2)
    return np.argmax(A2, axis=1)

def accuracy(Y_true, Y_pred):
    return np.mean(Y_true == Y_pred) * 100

# Train the model
input_size = 28 * 28
hidden_size = 128
output_size = 10
epochs = 100
learning_rate = 0.1

W1, b1, W2, b2 = train(x_train, y_train, input_size, hidden_size, output_size, epochs, learning_rate)

# Evaluate on test data
y_test_pred = predict(x_test, W1, b1, W2, b2)
y_test_true = np.argmax(y_test, axis=1)

test_acc = accuracy(y_test_true, y_test_pred)
print(f"Test Accuracy: {test_acc:.2f}%")


Epoch 0, Loss: 2.3031
Epoch 10, Loss: 2.2910
Epoch 20, Loss: 2.2692
Epoch 30, Loss: 2.2227
Epoch 40, Loss: 2.1304
Epoch 50, Loss: 1.9724
Epoch 60, Loss: 1.7470
Epoch 70, Loss: 1.4909
Epoch 80, Loss: 1.2614
Epoch 90, Loss: 1.0838
Test Accuracy: 78.79%
